In [3]:
import numpy as np 
import pandas as pd
import os 
import sys
from distutils.dir_util import copy_tree

# go to the directory where utility functions are stored
os.chdir(r'C:\Users\User\Desktop\WoodFrameBuildingTool\Python Tool Win\ModelAssembling')
from BuildingModelClass import BuildingModel
from utils import *
import subprocess
import json
from json import JSONEncoder
from os import spawnle
import win32api

In [4]:
# Utility function directory 
UtilDirectory = r'C:\Users\User\Desktop\WoodFrameBuildingTool\Python Tool Win\ModelAssembling'
# Base directory is the main directory that models, model inputs and utility directory stores
BaseDirectory = UtilDirectory.replace("ModelAssembling", "")
# Model directory is where you want to store your model
ModelDirectory = BaseDirectory + '\BuildingModels'
# DB directory is where you store Database.csv (for steel section)
DBDirecctory = UtilDirectory

# If there is no model directory, create one
if os.path.isdir(ModelDirectory) != True:
    os.chdir(BaseDirectory)
    os.mkdir('BuildingModels')
    
# Read in building name 
# Make sure the building name is consistent with the input folders
os.chdir(UtilDirectory)
with open('BuildingName.txt', 'r') as f:
    BuildingList = f.read() 
BuildingList = BuildingList.split('\n')

# Generate eigen analysis, pushover analysis and dynamic analysis models for each one in the building list
for i in range(len(BuildingList)):
    InfoDirectory = BaseDirectory + '\BuildingInfo\%s'%BuildingList[i]
    ModelClass = BuildingModel(BuildingList[i], InfoDirectory)

    # alternatively, one can choose to read inputs from json 
    ModelClass.read_in_txt_inputs(BuildingList[i], InfoDirectory)

    if os.path.isdir(ModelDirectory+'\%s'%BuildingList[i]) != True:
        os.chdir(ModelDirectory)
        os.mkdir('%s'%BuildingList[i])
        
    os.chdir(ModelDirectory+'/%s'%BuildingList[i])
    period = generateModalAnalysisModel(ModelClass.ID, ModelClass, BaseDirectory, DBDirecctory)
    # Turn off RunPushoverSwitch to speed up the model creation 
    generatePushoverAnalysisModel(ModelClass.ID, ModelClass, BaseDirectory, DBDirecctory,
                               GenerateModelSwitch = True, RunPushoverSwitch = False)
    generateDynamicAnalysisModel(ModelClass.ID, ModelClass, BaseDirectory, DBDirecctory, period,
                               GenerateModelSwitch = True)
    print(period)


[0.3522614619208698, 0.25412715353010734, 0.229149425056406, 0.10394190316450584]


In [5]:
def defineBaseReactionRecorders3DModel(ModelDirectory,BuildingModel,AnalysisType):
    os.chdir(ModelDirectory)
    
    with open('DefineBaseReactionRecorders3DModel.tcl', 'w') as tclfile:
        tclfile.write('# Define base node reaction recorders \n')
        tclfile.write('\n')
    numLeaningColumn = BuildingModel.leaningColumnNodesOpenSeesTags.shape[1]
    if 'Dynamic' in AnalysisType:
        tclfile.write('cd $pathToResults/EQ_$folderNumber/BaseReactions \n\n')
    elif 'Pushover' in AnalysisType:
        tclfile.write('cd $baseDir/$dataDir/BaseReactions \n\n')
        
    tclfile.write('# Base node vertical reactions \n')
    
    tclfile.write('recorder\tNode\t-file\tXPanelBaseNodesVerticalReactions.out\t-time\t-node\t')
    for i in range(BuildingModel.numberOfXDirectionWoodPanels[0]):
        tclfile.write('%i\t'%BuildingModel.XDirectionWoodPanelsBotTag[0,i])
    tclfile.write('-dof\t2\treaction\n')
    
    tclfile.write('recorder\tNode\t-file\tZPanelBaseNodesVerticalReactions.out\t-time\t-node\t')
    for i in range(BuildingModel.numberOfZDirectionWoodPanels[0]):
        tclfile.write('%i\t'%BuildingModel.ZDirectionWoodPanelsBotTag[0,i])
    tclfile.write('-dof\t2\treaction\n')
    
    tclfile.write('recorder\tNode\t-file\tLeaningColumnBaseNodeVerticalReactions.out\t-time\t-node\t')
    for i in range(numLeaningColumn):
        tclfile.write('%i\t'%BuildingModel.leaningColumnNodesOpenSeesTags[0,i])
    tclfile.write('-dof\t2\treaction\n')
    tclfile.write('\n')
    
    tclfile.write('# Base node horizontal reactions \n')
    
    tclfile.write('recorder\tNode\t-file\tXPanelBaseNodesHorizontalReactions.out\t-time\t-node\t')
    for i in range(BuildingModel.numberOfXDirectionWoodPanels[0]):
        tclfile.write('%i\t'%BuildingModel.XDirectionWoodPanelsBotTag[0,i])
    tclfile.write('-dof\t1\treaction\n')
    
    tclfile.write('recorder\tNode\t-file\tZPanelBaseNodesHorizontalReactions.out\t-time\t-node\t')
    for i in range(BuildingModel.numberOfZDirectionWoodPanels[0]):
        tclfile.write('%i\t'%BuildingModel.ZDirectionWoodPanelsBotTag[0,i])
    tclfile.write('-dof\t3\treaction\n')
    
    tclfile.write('recorder\tNode\t-file\tLeaningColumnBaseNodeXHorizontalReactions.out\t-time\t-node\t')
    for i in range(numLeaningColumn):
        tclfile.write('%i\t'%BuildingModel.leaningColumnNodesOpenSeesTags[0,i])
    tclfile.write('-dof\t1\treaction\n')
    
    tclfile.write('recorder\tNode\t-file\tLeaningColumnBaseNodeZHorizontalReactions.out\t-time\t-node\t')
    for i in range(numLeaningColumn):
        tclfile.write('%i\t'%BuildingModel.leaningColumnNodesOpenSeesTags[0,i])
    tclfile.write('-dof\t3\treaction\n')

    if BuildingModel.XRetrofitFlag:
